In [65]:
import numpy as np
from varname import nameof

# adapted from stack overflow
def bmatrix(a, var_name):
    """Stores in a file a LaTeX bmatrix from a numpy array."""
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join([f"{float(x):.6g}" for x in l.split()]) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    with open(f"./docs/aux-matrices/{var_name}.tex", 'w+') as f:
      f.write('\n'.join(rv))

In [66]:
x_1 = np.array([[1], [2]])
x_2 = np.array([[-1], [1]])
x_3 = np.array([[1], [0]])

mu_1 = np.array([[2], [2]])
mu_2 = np.array([[0], [0]])

Sigma_1 = np.array([[2, 1], [1, 2]])
Sigma_2 = np.array([[2, 0], [0, 2]])

bmatrix(x_1, nameof(x_1))
bmatrix(x_2, nameof(x_2))
bmatrix(x_3, nameof(x_3))

bmatrix(mu_1, nameof(mu_1))
bmatrix(mu_2, nameof(mu_2))

bmatrix(Sigma_1, nameof(Sigma_1))
bmatrix(Sigma_2, nameof(Sigma_2))

In [67]:
from scipy.stats import multivariate_normal

# because scipy is icky
mu_1_vector = np.array([2, 2])
mu_2_vector = np.array([0, 0])

p_k1 = 0.5
p_k2 = 0.5

p_x_1_given_k_1 = multivariate_normal(mu_1_vector, Sigma_1).pdf(x_1.T)
p_x_1_given_k_2 = multivariate_normal(mu_2_vector, Sigma_2).pdf(x_1.T)
p_x_2_given_k_1 = multivariate_normal(mu_1_vector, Sigma_1).pdf(x_2.T)
p_x_2_given_k_2 = multivariate_normal(mu_2_vector, Sigma_2).pdf(x_2.T)
p_x_3_given_k_1 = multivariate_normal(mu_1_vector, Sigma_1).pdf(x_3.T)
p_x_3_given_k_2 = multivariate_normal(mu_2_vector, Sigma_2).pdf(x_3.T)

for i in range(1, 4):
  for j in range(1, 3):
    print(f"p(x_{i} | C = k_{j}) = {eval(f'p_x_{i}_given_k_{j}'):0.6g}")

p(x_1 | C = k_1) = 0.0658407
p(x_1 | C = k_2) = 0.0227993
p(x_2 | C = k_1) = 0.00891057
p(x_2 | C = k_2) = 0.0482662
p(x_3 | C = k_1) = 0.0338038
p(x_3 | C = k_2) = 0.061975


In [68]:
posterior_x_1_k_1 = p_k1 * p_x_1_given_k_1
posterior_x_1_k_2 = p_k2 * p_x_1_given_k_2
normalized_posterior_x_1_k_1 = posterior_x_1_k_1 / (posterior_x_1_k_1 + posterior_x_1_k_2)
normalized_posterior_x_1_k_2 = posterior_x_1_k_2 / (posterior_x_1_k_1 + posterior_x_1_k_2)

posterior_x_2_k_1 = p_k1 * p_x_2_given_k_1
posterior_x_2_k_2 = p_k2 * p_x_2_given_k_2
normalized_posterior_x_2_k_1 = posterior_x_2_k_1 / (posterior_x_2_k_1 + posterior_x_2_k_2)
normalized_posterior_x_2_k_2 = posterior_x_2_k_2 / (posterior_x_2_k_1 + posterior_x_2_k_2)

posterior_x_3_k_1 = p_k1 * p_x_3_given_k_1
posterior_x_3_k_2 = p_k2 * p_x_3_given_k_2
normalized_posterior_x_3_k_1 = posterior_x_3_k_1 / (posterior_x_3_k_1 + posterior_x_3_k_2)
normalized_posterior_x_3_k_2 = posterior_x_3_k_2 / (posterior_x_3_k_1 + posterior_x_3_k_2)

print(f"p(C = k_1 | x_1) = {posterior_x_1_k_1:0.6g}")
print(f"p(C = k_2 | x_1) = {posterior_x_1_k_2:0.6g}")
print(f"NORMALIZED p(C = k_1 | x_1) = {normalized_posterior_x_1_k_1:0.6g}")
print(f"NORMALIZED p(C = k_2 | x_1) = {normalized_posterior_x_1_k_2:0.6g}")

print(f"p(C = k_1 | x_2) = {posterior_x_2_k_1:0.6g}")
print(f"p(C = k_2 | x_2) = {posterior_x_2_k_2:0.6g}")
print(f"NORMALIZED p(C = k_1 | x_2) = {normalized_posterior_x_2_k_1:0.6g}")
print(f"NORMALIZED p(C = k_2 | x_2) = {normalized_posterior_x_2_k_2:0.6g}")

print(f"p(C = k_1 | x_3) = {posterior_x_3_k_1:0.6g}")
print(f"p(C = k_2 | x_3) = {posterior_x_3_k_2:0.6g}")
print(f"NORMALIZED p(C = k_1 | x_3) = {normalized_posterior_x_3_k_1:0.6g}")
print(f"NORMALIZED p(C = k_2 | x_3) = {normalized_posterior_x_3_k_2:0.6g}")

p(C = k_1 | x_1) = 0.0329204
p(C = k_2 | x_1) = 0.0113997
NORMALIZED p(C = k_1 | x_1) = 0.742788
NORMALIZED p(C = k_2 | x_1) = 0.257212
p(C = k_1 | x_2) = 0.00445529
p(C = k_2 | x_2) = 0.0241331
NORMALIZED p(C = k_1 | x_2) = 0.155843
NORMALIZED p(C = k_2 | x_2) = 0.844157
p(C = k_1 | x_3) = 0.0169019
p(C = k_2 | x_3) = 0.0309875
NORMALIZED p(C = k_1 | x_3) = 0.352936
NORMALIZED p(C = k_2 | x_3) = 0.647064
